In [1]:
with open("input.txt", 'r') as f:
    input = f.read()

input1 = """Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0"""


# input example for part 2. Setting A=117440 will make the program output itself
####
# this program:
# 1. divides A by (2^3)
# 2. outputs A % 8
# 3. jumps to the beggining as long A!=0
####
# 0: iniA/8%8 = 0
# A = 0*8^1 + 3*8^2 + 5*8^3 + 4*8^4 + 3*8^5 = 117440
input1 = """Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0"""

# actual program to be tested
####
# this program:
# 1. (2,4) -> B = A % 8
# 2. (1,3) -> B = B XOR 3
# 3. (7,5) -> C = int(A // (2**B))
# 4. (1,5) -> B = B XOR 5
# 5. (0,3) -> A = int(A // (8))
# 6. (4,1) -> B = B XOR C
# 7. (5,5) -> out = B % 8
# 8. (3,0) -> jump to the beginning as long as A!=0
####
input1 = """Register A: 21539243
Register B: 0
Register C: 0

Program: 2,4,1,3,7,5,1,5,0,3,4,1,5,5,3,0"""

regs_prog = input.split("\n\n")
regs = regs_prog[0].split("\n")
a, b, c = int(regs[0].split(": ")[1]), int(regs[1].split(": ")[1]), int(regs[2].split(": ")[1])
PROG = regs_prog[1].split(": ")[1].split(",")
PROG = [int(x) for x in PROG]

A, B, C = a, b, c
A, B, C, PROG

(21539243, 0, 0, [2, 4, 1, 3, 7, 5, 1, 5, 0, 3, 4, 1, 5, 5, 3, 0])

In [2]:
IS_OPERAND_COMBO = {
    0: True,
    1: False,
    2: True,
    3: False,
    4: None,
    5: True,
    6: True,
    7: True
}

def operand_value(opcode, operand):
    if not IS_OPERAND_COMBO[opcode]: return operand
    # combo
    if operand in range(4): return operand
    if operand == 4: return A
    if operand == 5: return B
    if operand == 6: return C
    if operand == 7: raise ValueError

def execute(instruction_idx):
    # print(instruction_idx)
    global A, B, C, PROG
    # if instruction_idx > len(PROG): return -1
    opcode = PROG[instruction_idx]
    operand = PROG[instruction_idx+1]
    operand_val = operand_value(opcode, operand)
    
    out = None
    if opcode == 0: # division into A (combo)
        A = int(A // (2**operand_val))
    if opcode == 1: # bitwise XOR (literal)
        B = B ^ operand_val
    if opcode == 2: # modulo 8 (combo)
        B = operand_val % 8
    if opcode == 3: # jump (literal)
        if A != 0:
            instruction_idx = operand_val - 2 # -2 because we will increment later
    if opcode == 4: # bitwise XOR B & C (does not matter)
        B = B ^ C
    if opcode == 5: # outputs modulo 8 (combo)
        out = operand_val % 8
    if opcode == 6: # division but into B (combo)
        B = int(A // (2**operand_val))
    if opcode == 7: # division but into C (combo)
        C = int(A // (2**operand_val))
    instruction_idx += 2
    return instruction_idx, out

def execute_program():
    global PROG
    instruction_idx = 0
    outs = []
    while instruction_idx < len(PROG):
        instruction_idx2, out = execute(instruction_idx)
        # print(f"At instruction #{instruction_idx}. The instruction is {PROG[instruction_idx:instruction_idx+2]}, Registers {A,B,B},  output: {out}, next instruction #{instruction_idx2}")
        instruction_idx = instruction_idx2
        if out is not None: outs.append(out)
    return outs

# uncomment to uncover answer to part 1
# outs = execute_program()
# outs_str = [str(o) for o in outs]
# ",".join(outs_str)


### Part 2

In [3]:
from itertools import product
# what is outputed at always (B % 8)
# so at each time we want B%8 = prog[len-i]

def convert_to_int(list_of_coeffs):
    myA = 0
    for i, elem in enumerate((list_of_coeffs)):
    # for i, elem in enumerate(reversed(list_of_coeffs)):
        myA += elem*(8**i)
    # myA + tested*(8**len(saved))
    return myA


def find_magic_A():
    global PROG, A, B, C
    potentials_list = []

    for o in range(len(PROG)):
        # we start from least significant bit
        new_potentials_list = []
        if o == 0: potentials_list = [[]]
        for i in range(8):
            for saved in potentials_list:
                list_of_coeffs = [i] + saved
                A, B, C = convert_to_int(list_of_coeffs), b, c
                # print(A, B, C)
                outs = execute_program()
                # print(outs, PROG[-o-1:])
                if outs == PROG[-o-1:]:
                    new_potentials_list.append(list_of_coeffs)
        # print(len(new_potentials_list), new_potentials_list)
        potentials_list = new_potentials_list

    int_list = [convert_to_int(comb) for comb in potentials_list]
    # print(int_list)
    return min(int_list)


find_magic_A()

216549846240877